In [10]:
import requests
from pprint import pprint

## Objective
### To calculate some statistics related to AS path length.

#### First thing first, calculate data similar to what you did on the visualization tool

Use the following ground truth parameters:

In [12]:
base_url = ''
timestamp = 1204329600  # 2008-03-01
prefix4 = '202.12.27.33'  # M-root
prefix6 = '2001:dc3::35'

Get AS paths and peers for both IPv4 and IPv6 

In [16]:
def deduplicate(items):
    seen = set()
    for item in items:
        if item not in seen:
            yield item
            seen.add(item)

def get_peers(prefix, timestamp):
    url = 'https://stat.ripe.net/data/bgp-state/data.json?resource={0}&timestamp={1}'.format(prefix, timestamp)
    print('get_peers {}'.format(url))
    data = requests.get(url).json()
    data = data['data']['bgp_state']

    bgp_state = []
    if data:
        for item in data:
            route_info = {
                'peer': item['path'][0],
                'root': item['path'][-1],
                'as_path': list(deduplicate(item['path']))
            }
            bgp_state.append(route_info)

    peer = [item['peer'] for item in bgp_state]

    return bgp_state, peer


############################################################

bgp_state4, peer4 = get_peers(prefix4, timestamp)
bgp_state6, peer6 = get_peers(prefix6, timestamp)

get_peers https://stat.ripe.net/data/bgp-state/data.json?resource=202.12.27.33&timestamp=1204329600
get_peers https://stat.ripe.net/data/bgp-state/data.json?resource=2001:dc3::35&timestamp=1204329600


Find mutual peers

In [17]:
mutual_peers = list(set(peer4) & set(peer6))

Get mutual peers data

In [18]:
def get_peers_stat(peers, bgp_state4, bgp_state6):
    result = []
    for peer in sorted(peers):
        path4 = [d['as_path'] for d in bgp_state4 if d['peer'] == peer][0]
        path6 = [d['as_path'] for d in bgp_state6 if d['peer'] == peer][0]
        similar = 1 if path4 == path6 else 0
        temp_res = {
            'peer': peer,
            'similar': similar,
            'path4': path4,
            'path6': path6
        }
        result.append(temp_res)

    return result

##########################################################################

mutual_peers_stat = get_peers_stat(mutual_peers, bgp_state4, bgp_state6)

In [20]:
pprint(mutual_peers_stat)

[{'path4': [286, 3257, 7500],
  'path6': [286, 3257, 7500],
  'peer': 286,
  'similar': 1},
 {'path4': [559, 20965, 11537, 22388, 7660, 2500, 7500],
  'path6': [559, 20965, 2200, 7500],
  'peer': 559,
  'similar': 0},
 {'path4': [1280, 2497, 7500],
  'path6': [1280, 2500, 7500],
  'peer': 1280,
  'similar': 0},
 {'path4': [1853, 20965, 11537, 22388, 7660, 2500, 7500],
  'path6': [1853, 20965, 2200, 7500],
  'peer': 1853,
  'similar': 0},
 {'path4': [1916, 27750, 11537, 22388, 7660, 2500, 7500],
  'path6': [1916, 3549, 3257, 7500],
  'peer': 1916,
  'similar': 0},
 {'path4': [1930, 20965, 11537, 22388, 7660, 2500, 7500],
  'path6': [1930, 20965, 2200, 7500],
  'peer': 1930,
  'similar': 0},
 {'path4': [2497, 7500], 'path6': [2497, 7500], 'peer': 2497, 'similar': 1},
 {'path4': [2857, 8928, 2497, 7500],
  'path6': [2857, 6695, 3257, 7500],
  'peer': 2857,
  'similar': 0},
 {'path4': [2914, 2497, 7500],
  'path6': [2914, 7500],
  'peer': 2914,
  'similar': 0},
 {'path4': [3257, 7500], 'pa

#### Mutual peers

In [34]:
result_all = sorted([i['peer'] for i in mutual_peers_stat])
print(result_all)
print('total: {}'.format(len(result_all)))

[286, 559, 1280, 1853, 1916, 1930, 2497, 2857, 2914, 3257, 6667, 6762, 7575, 8447, 12779, 12859, 13237, 24875, 29449, 29636, 29686, 34225]
total: 22


#### Mutual peers with identical AS path

In [35]:
result_identical = sorted([i['peer'] for i in mutual_peers_stat if i['similar'] == 1])
print(result_identical)
print('total: {}'.format(len(result_identical)))

[286, 2497, 3257, 6667, 6762]
total: 5


#### Mutual peers with same AS path length but different AS path

In [39]:
result_diff = sorted([i['peer'] for i in mutual_peers_stat if len(i['path4']) == len(i['path6']) and i['similar'] == 0])
print(result_diff)
print('total: {}'.format(len(result_diff)))

[1280, 2857, 12859, 13237, 24875, 29449, 29636, 29686]
total: 8


#### Mutual peers that have longer IPv4 AS path

In [36]:
result_v4_longer = sorted([i['peer'] for i in mutual_peers_stat if len(i['path4']) > len(i['path6'])])
print(result_v4_longer)
print('total: {}'.format(len(result_v4_longer)))

[559, 1853, 1916, 1930, 2914, 7575, 12779]
total: 7


#### Mutual peers that have shorter IPv4 AS path

In [37]:
result_v4_shorter = sorted([i['peer'] for i in mutual_peers_stat if len(i['path4']) < len(i['path6'])])
print(result_v4_shorter)
print('total: {}'.format(len(result_v4_shorter)))

[8447, 34225]
total: 2
